In [1]:
import subprocess
import os
import sqlite3
import joblib
import re
import unicodedata
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, models
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

word_embedding_model = models.Transformer('neuralmind/bert-base-portuguese-cased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

C:\Users\vitor\.conda\envs\projeto_fernando_pessoa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def carregar_textos_duplos(db_path="textos.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT autor, texto, texto_semSW FROM textos")
    rows = cursor.fetchall()
    conn.close()

    textos_por_autor = {}
    textos_semSW_por_autor = {}

    for autor, texto, texto_semSW in rows:
        textos_por_autor.setdefault(autor, []).append(texto)
        textos_semSW_por_autor.setdefault(autor, []).append(texto_semSW)

    return textos_por_autor, textos_semSW_por_autor

In [3]:
def limpar_texto(texto):
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'[^a-zA-Z0-9À-ú,.!?;:\-\s]', '', texto)
    texto = unicodedata.normalize('NFKC', texto)
    return texto.strip()

In [4]:
def gerar_pkls_autores_duplo(textos_por_autor, textos_semSW_por_autor,
                             pasta_modelos='modelos_autores',
                             pasta_modelos_sw='modelos_autores_sw',
                             treino_ratio=0.8):
    """
    Gera PKLs para os textos normais e para os textos sem stopwords em pastas diferentes.
    """
    os.makedirs(pasta_modelos, exist_ok=True)
    os.makedirs(pasta_modelos_sw, exist_ok=True)

    todos_textos = []
    todos_textos_semSW = []

    for autor in textos_por_autor:
        textos = [limpar_texto(t) for t in textos_por_autor[autor]]
        textos_semSW = [limpar_texto(t) for t in textos_semSW_por_autor[autor]]

        textos_treino, textos_val = train_test_split(textos, train_size=treino_ratio, random_state=42)
        textos_semSW_treino, textos_semSW_val = train_test_split(textos_semSW, train_size=treino_ratio, random_state=42)

        # --- Com Stopwords ---
        joblib.dump({'autor': autor, 'textos': textos_treino}, os.path.join(pasta_modelos, f"{autor}_treino.pkl"))
        joblib.dump({'autor': autor, 'textos': textos_val}, os.path.join(pasta_modelos, f"{autor}_validacao.pkl"))
        for t in textos_treino:
            todos_textos.append({'autor': autor, 'texto': t})

        # --- Sem Stopwords ---
        joblib.dump({'autor': autor, 'textos': textos_semSW_treino},
                    os.path.join(pasta_modelos_sw, f"{autor}_treino.pkl"))
        joblib.dump({'autor': autor, 'textos': textos_semSW_val},
                    os.path.join(pasta_modelos_sw, f"{autor}_validacao.pkl"))
        for t in textos_semSW_treino:
            todos_textos_semSW.append({'autor': autor, 'texto': t})

        print(f"PKLs criados para {autor}: "
              f"comSW treino({len(textos_treino)}) / val({len(textos_val)}) | "
              f"semSW treino({len(textos_semSW_treino)}) / val({len(textos_semSW_val)})")

    # PKL geral
    joblib.dump(todos_textos, os.path.join(pasta_modelos, 'todos_autores_treino.pkl'))
    joblib.dump(todos_textos_semSW, os.path.join(pasta_modelos_sw, 'todos_autores_treino.pkl'))

    print(f"\nPKL geral criado: {len(todos_textos)} textos com SW, {len(todos_textos_semSW)} textos sem SW")


In [5]:
def treinar_embeddings(model_name, pasta_modelos='modelos_autores'):
    model = SentenceTransformer(model_name)
    for arquivo in os.listdir(pasta_modelos):
        if '_treino.pkl' in arquivo and 'todos_autores' not in arquivo:
            dados = joblib.load(os.path.join(pasta_modelos, arquivo))

            # Caso 1: dicionário com 'textos'
            if isinstance(dados, dict) and 'textos' in dados:
                textos = dados['textos']
                autor = dados['autor']

            # Caso 2: lista de dicionários (todos_autores_treino.pkl)
            elif isinstance(dados, list):
                textos = [d['texto'] for d in dados]
                autor = "desconhecido"

            else:
                raise ValueError(f"Formato inesperado no arquivo {arquivo}")

            embeddings = model.encode(textos, convert_to_numpy=True)
            joblib.dump(
                {'autor': autor, 'textos': textos, 'embeddings': embeddings},
                os.path.join(pasta_modelos, f"{autor}_{model_name.replace('/', '-')}_treino.pkl")
            )
    print(f"Treino concluído para modelo {model_name}")


In [6]:
def treinar_embeddings_geral(model_name, pasta_modelos='modelos_autores'):
    model = SentenceTransformer(model_name)
    pkl_geral = os.path.join(pasta_modelos, 'todos_autores_treino.pkl')

    if os.path.exists(pkl_geral):
        dados = joblib.load(pkl_geral)

        # Caso 1: lista de dicionários
        if isinstance(dados, list):
            textos = [d['texto'] for d in dados]
            autores = [d['autor'] for d in dados]

        # Caso 2: dicionário com listas
        elif isinstance(dados, dict):
            textos = dados.get('textos', [])
            autores = dados.get('autores', ["desconhecido"] * len(textos))
        else:
            raise ValueError("Formato inesperado no PKL geral")

        embeddings = model.encode(textos, convert_to_numpy=True)
        joblib.dump(
            {'autores': autores, 'textos': textos, 'embeddings': embeddings},
            os.path.join(pasta_modelos, f'todos_autores_{model_name.replace("/", "-")}_treino.pkl')
        )
        print(f"Embeddings gerais treinados para {model_name} com {len(textos)} textos")

In [7]:
def avaliar_modelo_completo(model_name, pasta_modelos='modelos_autores'):

    import numpy as np
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    import os
    import joblib

    model = SentenceTransformer(model_name)

    embeddings_por_autor = {}
    textos_por_autor = {}

    for arquivo in os.listdir(pasta_modelos):
        if '_treino.pkl' in arquivo and 'todos_autores' not in arquivo:
            dados = joblib.load(os.path.join(pasta_modelos, arquivo))
            autor = dados['autor']
            textos = dados['textos']
            textos_por_autor[autor] = textos_por_autor.get(autor, []) + textos

            embeddings = model.encode(textos, convert_to_numpy=True)
            if autor in embeddings_por_autor:
                embeddings_por_autor[autor] = np.vstack([embeddings_por_autor[autor], embeddings])
            else:
                embeddings_por_autor[autor] = embeddings

    # Classificação e contagem de acertos
    acertos = 0
    total = 0

    for autor_real, textos in textos_por_autor.items():
        for texto in textos:
            texto_embed = model.encode([texto], convert_to_numpy=True)
            similares = [(autor, cosine_similarity(texto_embed, emb).mean())
                         for autor, emb in embeddings_por_autor.items()]
            autor_predito = max(similares, key=lambda x: x[1])[0]
            if autor_predito == autor_real:
                acertos += 1
            total += 1

    acuracia = acertos / total if total > 0 else 0
    print(f"Acurácia do modelo {model_name}: {acuracia * 100:.2f}%")
    return acuracia

In [8]:
def selecionar_melhor_modelo_completo(modelos, pasta_modelos='modelos_autores'):
    resultados = {}
    for modelo in modelos:
        print(f"\nAvaliando modelo: {modelo}")
        acc = avaliar_modelo_completo(modelo, pasta_modelos)
        resultados[modelo] = acc

    melhor_modelo = max(resultados, key=resultados.get)
    print(f"\nMelhor modelo: {melhor_modelo} com acurácia {resultados[melhor_modelo] * 100:.2f}%")
    return melhor_modelo

In [9]:
# Exemplo de uso
textos_por_autor, textos_semSW_por_autor = carregar_textos_duplos("textos.db")
gerar_pkls_autores_duplo(textos_por_autor, textos_semSW_por_autor)





modelos_teste = [
    # 'paraphrase-multilingual-MiniLM-L12-v2',
    # 'all-MiniLM-L6-v2',
    # 'distiluse-base-multilingual-cased-v2',
    # 'neuralmind/bert-base-portuguese-cased',
    # 'neuralmind/bert-large-portuguese-cased', Não foi bom
    # 'all-MiniLM-L12-v2',
    'paraphrase-multilingual-mpnet-base-v2'
]

# melhor_modelo = selecionar_melhor_modelo_completo(modelos_teste)
# melhor_modelo = selecionar_melhor_modelo_completo(modelos_teste,"projeto_fernando_pessoa/modelos_autores_sw" )
melhor_modelo = 'paraphrase-multilingual-mpnet-base-v2'

PKLs criados para Ricardo Reis: comSW treino(214) / val(54) | semSW treino(214) / val(54)
PKLs criados para Bernardo Soares: comSW treino(325) / val(82) | semSW treino(325) / val(82)
PKLs criados para Álvaro de Campos: comSW treino(302) / val(76) | semSW treino(302) / val(76)
PKLs criados para Alberto Caeiro: comSW treino(101) / val(26) | semSW treino(101) / val(26)

PKL geral criado: 942 textos com SW, 942 textos sem SW


In [10]:
# Treinar embeddings na pasta com stopwords
treinar_embeddings("paraphrase-multilingual-mpnet-base-v2", pasta_modelos="modelos_autores")

# Treinar embeddings na pasta sem stopwords
treinar_embeddings("paraphrase-multilingual-mpnet-base-v2", pasta_modelos="modelos_autores_sw")

# Treino geral
treinar_embeddings_geral("paraphrase-multilingual-mpnet-base-v2", pasta_modelos="modelos_autores")
treinar_embeddings_geral("paraphrase-multilingual-mpnet-base-v2", pasta_modelos="modelos_autores_sw")


Treino concluído para modelo paraphrase-multilingual-mpnet-base-v2
Treino concluído para modelo paraphrase-multilingual-mpnet-base-v2
Embeddings gerais treinados para paraphrase-multilingual-mpnet-base-v2 com 942 textos
Embeddings gerais treinados para paraphrase-multilingual-mpnet-base-v2 com 942 textos


In [11]:
def classificar_texto(texto, melhor_modelo, pasta_modelos='modelos_autores'):

    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    import joblib
    import os
    import numpy as np

    model = SentenceTransformer(melhor_modelo)
    embedding_texto = model.encode([texto], convert_to_numpy=True)

    # Carregar embeddings de todos os autores gerados com o melhor modelo
    modelo_nome_arquivo = melhor_modelo.replace('/', '-')
    pkls_modelo = [f for f in os.listdir(pasta_modelos) if modelo_nome_arquivo in f and '_treino.pkl' in f and 'todos_autores' not in f]

    resultados = []
    for arquivo in pkls_modelo:
        dados = joblib.load(os.path.join(pasta_modelos, arquivo))
        sim = cosine_similarity(embedding_texto, dados['embeddings']).mean()
        resultados.append((dados['autor'], sim))

    resultados.sort(key=lambda x: x[1], reverse=True)
    autor_predito = resultados[0][0] if resultados else None

    return autor_predito, resultados


In [52]:
def interpretar_texto_ollama(texto_a_interpretar, autor_predito='todos_autores', pasta_modelos='modelos_autores_sw', modelo_llm='phi3'):
    """
    Interpreta um texto de acordo com o estilo do autor usando o Ollama CLI local.
    """
    try:
        dados = joblib.load(os.path.join(pasta_modelos, f"{autor_predito}_paraphrase-multilingual-mpnet-base-v2_treino.pkl"))
        textos_exemplo = dados.get('textos', [])
        exemplo_texto = textos_exemplo[0][:500] if textos_exemplo else "Nenhum exemplo de texto disponível para este autor."
    except FileNotFoundError:
        exemplo_texto = "Nenhum exemplo de texto disponível para este autor."
        print(f"Aviso: Arquivo de modelo para o autor '{autor_predito}' não encontrado.")

    prompt = f"""Você é {autor_predito}. Reescreva o texto abaixo mantendo o sentido original.
Responda em português.

Texto: "{texto_a_interpretar}"
Exemplo: "{exemplo_texto}"
"""

    try:
        result = subprocess.run(
            ["ollama", "run", modelo_llm],
            input=prompt,
            capture_output=True,
            text=True,
            encoding='utf-8'
        )
    except FileNotFoundError:
        print("Erro: O comando 'ollama' não foi encontrado. Certifique-se de que o Ollama está instalado e no seu PATH.")
        return None

    if result.returncode != 0:
        print(f"\n--- Erro do Ollama ---")
        print(f"Código de retorno: {result.returncode}")
        print(f"Saída de erro: {result.stderr}")
        raise RuntimeError(f"Erro ao chamar Ollama: {result.stderr}")

    if result.stdout is None:
        return ""

    return result.stdout.strip()


In [53]:
texto_teste = """No entardecer dos dias de Verão, às vezes,
Ainda que não haja brisa nenhuma, parece
Que passa, um momento, uma leve brisa...
Mas as árvores permanecem imóveis."""

autor_predito, todas_similaridades = classificar_texto(texto_teste, melhor_modelo)



print(f"Autor mais provável: {autor_predito}\n")
print("Similaridade com cada autor:")
for autor, sim in todas_similaridades:
    print(f"{autor}: {sim:.2f}")


interpretacao = interpretar_texto_ollama(texto_teste, autor_predito)
print(f"\n--- Interpretação pelo LLM ---\n{interpretacao}")

Autor mais provável: Alberto Caeiro

Similaridade com cada autor:
Alberto Caeiro: 0.34
Ricardo Reis: 0.33
Bernardo Soares: 0.29
Álvaro de Campos: 0.29

--- Interpretação pelo LLM ---
No amanhecer daquele calor intenso do Verão,
Às vezes, mesmo sem uma breve brisa desfilar,  
Parece que flui um murmúrio leve... mas as árvores têm sua eternidade imutável.
